## Modelle auswerten

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
print('Pakete erfolgreich importiert.')

In [ ]:
size = [25, 32, 40, 48, 55] # Pizza-Größen in cm
price = [3.28, 4.33, 5.75, 6.10, 6.15] # Preise
df = pd.DataFrame({'size':size, 'price':price})
df

Auf lineare Korrelation prüfen:

In [ ]:
sns.regplot(x='size',y='price',data=df)
plt.show()

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr(), annot=True)
plt.show()

Um die genaue Korrelationen beschreiben zu können, brauchen wir ein Modell der Klasse LinearRegression:

In [25]:
reg = LinearRegression()

Das Modell trainieren:

- mit dem ganzen Dataset trainieren
- Daten in Train- und Test-set einteilen

Wir können entweder unser Modell mit ganzen Daten trainieren und eben da noch evaluieren, oder die Daten in zwei Subsets, nämlich Train-Set und Test-Set einteilen. Dann das Modell mit Train-Set trainieren und mit Test-Set evaluieren.

Weil jetzt unser Dataset viel zu klein ist (nur 5 Einträge), trainieren wir das Modell mit dem ganzen Dataset:

In [ ]:
X = df[['size']]
y = df['price']
reg.fit(X,y)

In [ ]:
features = df[['size']]
reg.predict(features)

In [ ]:
df['prediction'] = reg.predict(features)
df

### Wie ist überhaupt 'Genauigkeit' beschreibbar?

In der Statistik wird Genauigkeit anhand der sogenannten Maßzahlen beschrieben.  
Jede Maßzahl eignet sich für gewisse Modelle und wird anhand eines Algorithmus Berechnet. 
### Mittlerer absoluter Fehler
Eine der Maßzahlen in der Statistik heißt _mean absolut error_. (Deutsch: Mittlerer absoluter Fehler). Der mittlere absolute Fehler ist wie folgt definiert:
<img src='https://wikimedia.org/api/rest_v1/media/math/render/svg/d19c272643189496e85c66d8c9722b515d3b39ac' />
wobei folgende Variablen Verwendung finden:

- Anzahl der Vorhersagewerte(n)
- Vorhersagewerte
- Beobachtungswerte (Realität)

[Quelle](https://de.wikipedia.org/wiki/Mittlerer_absoluter_Fehler)


Im Folgenden berechnen wir diese Maßzahl für unser aktuelles Modell:

In [29]:
# Wir berechnen die Differenzen:
dif = df['price'] - df['prediction']
difs = dif.abs() # Absoluter Wert von Elementen (unabhängig von Richtung/Vorzeichen)

Dann berechnen wir den Durchschnitt dieser Differenzen:

In [ ]:
difs.mean()

Wir kommen auf ca. `0.325666435986159`. Das heißt, die Einschätzung unseres Modells liegt meistens ca. `0.33` über oder unter der Realität.

### R²-Wert (R-Quadrat) berechnen
Eine weitere wichtige Maßzahl (engl. metric) ist dabei der sogenannte R-Quadrat $R^2$, den wir mit Hilfe der Methode `score()` für lineare Regressionsmodelle berechnen. 

In [ ]:
reg.score(X,y)

Die Methode `score()` liefert den sogenannten R-Quadrat-Wert zurück. Dieser Wert liegt zwischen `0.0` und `1.0`. Je dieser Wert `1.0` näher steht, desto _genauer_ ist das Modell.

[Bestimmtheitsmaß bei Wiki](https://de.wikipedia.org/wiki/Bestimmtheitsma%C3%9F)

Jede Maßzahl eignet sich für gewisse Algorithmen. In unserem Beispiel, also Single-Linear-Regression (lineare Regression mit einem einzelnen Feature) eignet sich R-Squared sehr gut, um die Genauigkeit des Modells zu beschreiben. Daher die Methode `score()`

#### Mean-Squared-Error (MSE) oder Mittlere quadratische Abweichung

wird von Mittlerem-Absolut-Fehler abgeleitet [Quelle](https://de.wikipedia.org/wiki/Mittlere_quadratische_Abweichung).

MSE = $\frac{1}{n} \Sigma_{i=1}^n({y}-\hat{y})^2$

Im Folgenden berechnen wir diese Abweichung für unser Modell:

In [ ]:
df

In [ ]:
pow((df['price'] - df['prediction']),2).sum()/df.shape[0]

Scikitlearn verfügt in Submodul `metrics` über Methoden und Funktionen, die diese Maßzahlen direkt berechnen können:

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
mse = mean_squared_error(df['price'], df['prediction'])
mae = mean_absolute_error(df['price'], df['prediction'])
rmse = root_mean_squared_error(df['price'], df['prediction'])
print(f"MAE: {mae}", f"MSE: {mse}", f"RMSE: {rmse}", sep="\n")

### p-Wert (engl. Pearson Value)
ist noch eine weitere maßgebende Zahl in Statistik, bzw. Machine Learning. Im Deutschen heißt es `Korrelationskoeffizient` [Quelle](https://de.wikipedia.org/wiki/Korrelationskoeffizient)
Diese Zahl wird anhand eines sogenannten Pearson-Algorithmus berechnet und wie folgend interpretiert:

- `p < 0.001` spricht für eine _starke_ Korrelation
- `0.001 < p < 0.05`: spricht für eine _moderate_ Korrelation
- `0.05 < p < 0.1`: spricht für eine _schwache_ Korrelation
- `0.1 < p`: spricht für _keine_ Korrelation

zwischen Attributen. Den p-Wert kann man direkt aus `scipy.stats` berechnen lassen:

In [ ]:
from scipy.stats import pearsonr
# In wiefern besteht eine Korrelation
# zwischen size und price?
pearsonr(df['size'],df['price'])

Die zwei Zahlen im Tupel sind jeweils ein Pearson-Koeffizient und ein p-Wert.  
Wir können damit eine Funktion bilden, die auf Basis der Größe (size) den P-Wert berechnet:

In [53]:
def correl(x,y):
    p = pearsonr(df[x],df[y])[1]
    if p < 0.001:
        return 'Strong'
    elif 0.001 < p < 0.05:
        return 'Moderate'
    elif 0.05 < p < 0.1:
        return 'Weak'
    else:
        return 'No way!'

In [ ]:
correl('size', 'price')

In unserem Beispiel besteht eine _moderate_ lineare Korrelation zwischen `size` und `price`

Weitere Metrics werden wir im Rahmen von Projekten und Aufgaben kennenlernen, u.a. F-Score und Recall.